<a href="https://colab.research.google.com/github/raurica/Image-Forgery/blob/main/ImageForgery_not_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import os, shutil
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from zipfile import ZipFile

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [56]:
import random
def tf_seed(seed=0):
  np.random.seed(seed)  # numpy seed
  tf.random.set_seed(seed)  # tensorflow seed
  random.seed(seed)  # random seed
  os.environ['TF_DETERMINISTIC_OPS'] = "1"
  os.environ['TF_CUDNN_DETERMINISM'] = "1"
  os.environ['PYTHONHASHSEED'] = str(seed)

tf_seed()

##Loading the dataset

In [61]:
dataset = "MICC-F220" #@param ["MICC-F220", "MICC-F600", "MICC-F2000", "Merged datasets"]
id_data = ['1pL6W3mN931TmmaCtMZsR0z1YuVfkvH4J','1JDz-suK2Qy0HDTXzq8XPDRs5QmRX9e4s','1G5XjcONn5-TK5h_S3sPNwYxDICNWZ-8h']
names_dataset = ["MICC-F220", "MICC-F600", "MICC-F2000"]

if dataset=='MICC-F220':
  fileId = drive.CreateFile({'id':id_data[0]})
elif dataset=='MICC-F600':
  fileId = drive.CreateFile({'id':id_data[1]})
elif dataset=='MICC-F2000':
  fileId = drive.CreateFile({'id':id_data[2]})
elif dataset=='Merged datasets':
  for i in range(len(id_data)):
    fileId = drive.CreateFile({'id':id_data[i]})
    fileId.GetContentFile(fileId['title'])
    name_ds = names_dataset[i]+'.zip'
    print(name_ds)
    #!unzip -q name_ds
    ZipFile(names_dataset[i]+'.zip').extractall('/content/')


  # folder in which all the content will be merged
  merge_folder = ["/content/Datasets/test_train_images/original",
                "/content/Datasets/test_train_images/tampered",
                "/content/Datasets/validation_images/original",
                "/content/Datasets/validation_images/tampered"]

  if not os.path.exists('/content/Datasets/'):
    os.mkdir('/content/Datasets/')
    os.mkdir('/content/Datasets/test_train_images/')
    os.mkdir('/content/Datasets/validation_images/')

    for folder in merge_folder:
      if not os.path.exists(folder):
        os.mkdir(folder)

  # current folder path
  current_folder = os.getcwd() 
    
  # list of folders to be merged
  list_dir = [['/content/MICC-F220/test_train_images/original', '/content/MICC-F600/test_train_images/original','/content/MICC-F2000/test_train_images/original'], 
              ['/content/MICC-F220/test_train_images/tampered', '/content/MICC-F600/test_train_images/tampered','/content/MICC-F2000/test_train_images/tampered'],
              ['/content/MICC-F220/validation_images/original', '/content/MICC-F600/validation_images/original','/content/MICC-F2000/validation_images/original'],
              ['/content/MICC-F220/validation_images/tampered', '/content/MICC-F600/validation_images/tampered','/content/MICC-F2000/validation_images/tampered']]
    
  # enumerate on list_dir to get the content of all the folders ans store it in a dictionary
  content_list = {}
  for i in range(len(merge_folder)):
    for index, val in enumerate(list_dir[i]):
        path = os.path.join(current_folder, val)
        content_list[ list_dir[i][index] ] = os.listdir(path)

    # Function to create new folder if not exists
    def make_new_folder(folder_name, parent_folder_path):
        path = os.path.join(parent_folder_path, folder_name)
        try: 
            mode = 0o777
            os.mkdir(path, mode) 
        except OSError as error: 
            print(error)
              

      
    # merge_folder path - current_folder + merge_folder
    merge_folder_path = os.path.join(current_folder, merge_folder[i]) 
      
    # create merge_folder if not exists
    make_new_folder(merge_folder[i], current_folder)


    # loop through the list of folders
    for sub_dir in content_list:
        # loop through the contents of the list of folders
        for contents in content_list[sub_dir]:
      
            # make the path of the content to move 
            path_to_content = sub_dir + "/" + contents  
            if os.path.exists(path_to_content):
              # make the path with the current folder
              dir_to_move = os.path.join(current_folder, path_to_content )
              filename = os.path.basename(path_to_content)
              dest = os.path.join(merge_folder_path,filename)
              shutil.move(dir_to_move, dest)
  for ds in names_dataset:    
    shutil.rmtree('/content/'+ds)   


if not dataset=='Merged datasets':
  fileId.GetContentFile(fileId['title'])
  ZipFile(dataset+'.zip').extractall('/content/')

print('Uploaded {}, id {}'.format(fileId['title'],fileId['id']))

Uploaded MICC-F220.zip, id 1pL6W3mN931TmmaCtMZsR0z1YuVfkvH4J


##Main variables

In [62]:
EPOCHS = 15 #@param ["15", "25", "35", "50", "75", "100"] {type:"raw"}
BATCH_SIZE = 16 #@param {type:"integer"}

IMAGE_SIZE = (224,224)

##Creating labels

In [63]:
class_names=['original','tampered']
class_name_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)
print(class_name_label)

{'original': 0, 'tampered': 1}


##Load data

In [64]:
def load_data():

    tf_seed()

    if not dataset=='Merged datasets':
      DIRECTORY = r"/content/"+dataset
    else:
      DIRECTORY = r"/content/Datasets"

    CATEGORY = ["test_train_images","validation_images"]
    
    output = []
    
    for category in CATEGORY:
        path = os.path.join(DIRECTORY, category)
        images = []
        labels = []
        
        print("Loading {}".format(category))
        
        for folder in os.listdir(path):
          if not folder.startswith('.'):           
            label = class_name_label[folder]
            
            for file in os.listdir(os.path.join(path, folder)): 
                
                img_path = os.path.join(os.path.join(path,folder),file)
                print(img_path)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)
                
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype="float32")
        labels = np.array(labels, dtype="float32")
    return images,labels #output

In [65]:
save_plot = False #@param {type:"boolean"}

def plot_accuracy_loss(H):

  plt.style.use("ggplot")
  plt.figure()
  fig=plt.figure(figsize=(12,7))

  new_epochs=len(H.history['loss'])
  plt.plot(np.arange(0, new_epochs), H.history["loss"], label="train_loss")
  plt.plot(np.arange(0, new_epochs), H.history["val_loss"], label="val_loss")
  plt.plot(np.arange(0, new_epochs), H.history["accuracy"], label="train_acc")
  plt.plot(np.arange(0, new_epochs), H.history["val_accuracy"], label="val_acc")
  plt.title("Training Loss and Accuracy on Dataset")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss/Accuracy")
  plt.legend(loc="lower left")

  if save_plot==True:
    plt.savefig('ImageForgery_'+dataset+'_'+str(EPOCHS)+'ep.jpg')
  plt.show()


In [66]:
images, labels = load_data()
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2) 

Loading test_train_images
/content/MICC-F220/test_train_images/original/CRW_4810_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1568_scale.jpg
/content/MICC-F220/test_train_images/original/CRW_4838_scale.jpg
/content/MICC-F220/test_train_images/original/CRW_4836_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1572_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1561_scale.jpg
/content/MICC-F220/test_train_images/original/CRW_4840_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1570_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1557_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1580_scale.jpg
/content/MICC-F220/test_train_images/original/CRW_4830_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1575_scale.jpg
/content/MICC-F220/test_train_images/original/DSC_1549_scale.jpg
/content/MICC-F220/test_train_images/original/CRW_4833_scale.jpg
/content/MICC-F220/test_train_images/original/CRW_4827_scale.jpg

##Preprocessing

In [67]:
# Normalize data
train_images = train_images / 255
test_images = test_images / 255

##Model and training

In [68]:
import torch, psutil

class MemoryUsage(keras.callbacks.Callback):
   def __init__(self):
      # setting device on GPU if available, else CPU
      self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      print('Using device for training:', self.device)
      self.max_RAM=[]
      self.max_GPU=[]

   def get_memory_usage(self):
      gpu_dict = tf.config.experimental.get_memory_info('GPU:0')
      print('\n GPU memory details [current: {} gb, peak: {} gb]'.format(
          float(gpu_dict['current']) / (1024 ** 3), 
          float(gpu_dict['peak']) / (1024 ** 3)))
   
   def get_size(self, byte, suffix="GB"):
    factor = 1024
    
    for unit in ["", "K", "M", "GB", "T", "P"]:
        if byte < factor:
            return f"{byte:.2f} GB"
        byte /= factor

   def on_train_end(self, epoch, logs=None):
      i=np.argmax(self.max_RAM)
      j=np.argmin(self.max_RAM)
      self.get_memory_usage()
      print("MAX RAM USAGE: %s / %s (%s)" % (self.get_size(self.max_RAM[i][0]), self.get_size(self.max_RAM[i][1]), str(self.max_RAM[i][2]) + "%" ))
    
   def on_epoch_end(self,epoch,logs=None):
      svmem = psutil.virtual_memory()
      self.max_RAM.append((svmem.active, svmem.total, svmem.percent))
      #self.get_memory_usage()
      

In [71]:
save_model = True #@param {type:"boolean"}

tf_seed()

#============ DEFINE MODEL ===============

model = Sequential()

model.add(Conv2D(16, 1, activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3),1, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3),1, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256, (3,3),1, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(512, (3,3),1, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(2,activation='softmax'))

model.compile(optimizer="adam", 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

#============ TRAINING MODEL ===============

import time
start_time = time.time()

model.summary()

H = model.fit(train_images, train_labels, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS,
                    validation_data=(test_images,test_labels),
                    #callbacks = [memory_usage, callback_early_stopping],
                    shuffle=True,
                    verbose=1
                    )

#============ EVALUATE MODEL ===============

model.evaluate(test_images, test_labels)

print("TRAINING TIME","--- %s seconds ---" % round((time.time() - start_time),3))


#============ SAVING MODEL ===============

if save_model==True: model.save('ImageForgery_'+dataset+'_'+str(EPOCHS)+'ep.h5') 

[INFO] compiling model...
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 224, 224, 16)      64        
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 112, 112, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 110, 110, 32)      4640      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 55, 55, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 53, 53, 64)        18496     
                                                                 
 max_pooling2d_20 (MaxPoolin

In [ ]:
plot_accuracy_loss(H)

In [48]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
import itertools
from itertools import cycle
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

predictions = model.predict(test_images)
pred_labels = np.argmax(predictions, axis=1)

print(classification_report(test_labels, pred_labels))

print(confusion_matrix(test_labels,pred_labels))
tn, fp, fn, tp = confusion_matrix(list(test_labels), list(pred_labels), labels=[0, 1]).ravel()

print('TP', tp,'\nTN', tn,'\nFP', fp,'\nFN', fn)

print('\nTPR', round(tp/(tp+fn),4))
print('FNR', round(fn/(fn+tp),4))
print('FPR', round(fp/(fp+tn),4))
print('TNR', round(tn/(tn+fp),4))
print('\nAccuracy:',round(accuracy_score(test_labels, pred_labels),4))
print('F1-score', round(2*tp/(2*tp+fp+fn),3))


              precision    recall  f1-score   support

         0.0       0.95      0.91      0.93       138
         1.0       0.88      0.93      0.90       101

    accuracy                           0.92       239
   macro avg       0.91      0.92      0.91       239
weighted avg       0.92      0.92      0.92       239

[[125  13]
 [  7  94]]

True Positive 94 
True Negative 125 
False Positive 13 
False Negative 7

TPR 0.9307
FNR 0.0693
FPR 0.0942
TNR 0.9058

Accuracy: 0.9163
F1-score 0.904
